In [4]:
"""
🎯 학습 목표: ConversationBufferMemory와 듀얼 체인 아키텍처를 활용한 지능형 영화 이모지 시스템 구축
📚 핵심 개념: Memory 관리, Strategy Pattern, 지능적 라우팅, FewShotChatMessagePromptTemplate
🚀 실행 결과: 영화를 3개 이모지로 변환하고 대화 순서를 정확히 기억하는 AI 시스템

📋 과제 요구사항:
- 영화 제목 → 3개 이모지 변환 (예: "탑건" → "🛩️👨‍✈️🔥")
- FewShotChatMessagePromptTemplate으로 일관된 이모지 형식 보장  
- ConversationBufferMemory로 대화 순서 추적
- "첫 번째로 물어본 영화가 무엇인가요?" 같은 메모리 질문에 정확한 답변
"""

# 🔧 필수 모듈 임포트
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory

# === 1단계: LLM 초기화 ===
# 🧠 개념: 이모지 생성의 일관성을 위해 낮은 temperature 설정
chat = ChatOpenAI(
    temperature=0.1  # 📌 용도: 일관된 이모지 패턴 생성 보장, 타입: float
)

# === 2단계: Few-shot 학습용 예시 데이터 준비 ===
# 🧠 개념: 문화적으로 적절하고 직관적인 이모지 조합 패턴 학습
# 💡 전략: 장르별 다양성 + 한국/해외 영화 균형 + 명확한 상징성
examples = [
    {
        "movie": "탑건", 
        "emojis": "🛩️👨‍✈️🔥"
        # 📌 패턴 분석: 주요 소재(전투기) + 인물(조종사) + 감정(열정/액션)
    },
    {
        "movie": "대부", 
        "emojis": "👨‍👨‍👦🔫🍝"
        # 📌 패턴 분석: 가족 관계 + 마피아 상징 + 이탈리아 문화
    },
    {
        "movie": "타이타닉", 
        "emojis": "🚢💑🌊"
        # 📌 패턴 분석: 주요 배경(배) + 관계(로맨스) + 사건(침몰/바다)
    },
    {
        "movie": "매트릭스", 
        "emojis": "🕶️💊🤖"
        # 📌 패턴 분석: 상징적 소품 + 핵심 선택 + 주제(AI/가상현실)
    }
]

# === 3단계: 영화 이모지 생성 체인 구성 ===
# 🧠 개념: 창의적 변환 작업에 특화된 전문 체인
movie_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 영화 전문 이모지 변환 AI입니다.

🎯 역할: 영화 제목을 그 영화의 핵심을 나타내는 정확히 3개의 이모지로 변환

📋 변환 규칙:
1. 반드시 3개의 이모지만 사용 (더 많거나 적으면 안됨)
2. 영화의 가장 특징적인 요소들을 선택
3. 장르, 주요 소재, 감정을 균형있게 표현
4. 문화적으로 적절하고 직관적인 이모지 선택

⚠️ 응답 형식: 텍스트 설명 없이 이모지만 응답하세요"""),
    
    # Few-Shot 학습 부분
    FewShotChatMessagePromptTemplate(
        example_prompt=ChatPromptTemplate.from_messages([
            ("human", "영화: {movie}"),
            ("assistant", "{emojis}")
        ]),
        examples=examples  # 📌 용도: 일관된 이모지 패턴 학습
    ),
    
    ("human", "영화: {movie}")
])

# === 4단계: 메모리 조회 체인 구성 ===
# 🧠 개념: 정확한 순서 분석에 특화된 전문 체인
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 대화 기록 전문 분석가입니다.

🎯 역할: 대화 기록을 정확하게 분석하여 영화 질문 순서를 파악

📋 분석 규칙:
1. "영화: [제목]" 패턴의 질문만 순서에 포함
2. 메모리 조회 질문과 응답은 순서에서 제외  
3. 시간 순서대로 정확히 파악
4. 동일한 영화가 여러 번 나와도 각각을 별개 순서로 계산

🔍 순서 정의:
• "첫 번째" = 가장 처음 물어본 영화
• "두 번째" = 두 번째로 물어본 영화
• "마지막" = 가장 최근에 물어본 영화"""),
    
    ("system", "대화 기록:\n{chat_history}"),
    ("human", "{question}")
])

# === 5단계: 메모리 시스템 초기화 ===
# 🧠 개념: ConversationBufferMemory로 완전한 대화 기록 보존
memory = ConversationBufferMemory(
    return_messages=True  # 📌 용도: 메시지 객체 형태 반환으로 구조화된 데이터 활용
)

# === 6단계: 듀얼 체인 아키텍처 구성 ===  
# 🧠 개념: Strategy Pattern - 입력에 따른 전문화된 처리 전략
movie_chain = movie_prompt | chat     # 📌 용도: 창의적 이모지 변환 전문
memory_chain = memory_prompt | chat   # 📌 용도: 정확한 순서 분석 전문

# === 7단계: 지능적 라우팅 함수 ===
# 🧠 개념: 입력 분석으로 적절한 체인 선택하는 통합 인터페이스
def ask(input_text):
    """
    📋 기능: 입력에 따라 영화 이모지 생성 또는 메모리 조회 수행
    🏗️ 아키텍처: Strategy Pattern 기반 지능적 라우팅
    📥 입력: 영화제목 또는 메모리 질문
    📤 출력: 이모지 3개 또는 기억된 영화 정보
    """
    
    # 메모리 조회 패턴 감지
    memory_keywords = ["물어본 영화", "첫 번째", "두 번째", "세 번째", "마지막"]
    is_memory_query = any(keyword in input_text for keyword in memory_keywords)
    
    if is_memory_query:
        # 🧠 메모리 조회 경로 - 대화 기록 분석
        chat_history = "\n".join([
            f"{'사용자' if msg.type == 'human' else 'AI'}: {msg.content}"
            for msg in memory.chat_memory.messages
        ]) if memory.chat_memory.messages else "대화 기록 없음"
        
        result = memory_chain.invoke({
            "chat_history": chat_history,
            "question": input_text
        })
        
        return result.content
        
    else:
        # 🎨 영화 이모지 생성 경로 - 창의적 변환 + 메모리 저장
        result = movie_chain.invoke({"movie": input_text})
        
        # 선택적 메모리 저장 (영화 요청만 저장, 메모리 조회는 저장 안함)
        memory.chat_memory.add_user_message(f"영화: {input_text}")
        memory.chat_memory.add_ai_message(result.content)
        
        return result.content

# === 8단계: 실전 테스트 실행 ===
# 🚀 실행: 듀얼 체인 시스템의 종합 테스트

print("=== Challenge 03: 듀얼 체인 메모리 시스템 테스트 ===")

# 영화 이모지 생성 테스트
print("\n🎬 영화 이모지 생성 테스트:")
print(f"인셉션 → {ask('인셉션')}")
print(f"어벤져스 → {ask('어벤져스')}")  
print(f"명량 → {ask('명량')}")

# 메모리 조회 테스트  
print("\n🧠 메모리 조회 테스트:")
print(f"첫 번째 영화: {ask('첫 번째로 물어본 영화가 무엇인가요?')}")
print(f"마지막 영화: {ask('마지막으로 물어본 영화가 무엇인가요?')}")

# 추가 영화와 복잡한 순서 추적
ask("기생충")
print(f"두 번째 영화: {ask('두 번째로 물어본 영화가 무엇인가요?')}")

print("\n✅ 테스트 완료!")

# 💡 실무 확장 아이디어:
# - 다국어 영화 지원: 언어별 문화적 적절성 고려한 이모지 선택
# - 장르별 특화: 호러, 코미디, 드라마별 최적화된 이모지 전략  
# - 품질 평가: 생성된 이모지의 적절성을 자동 평가하는 시스템
# - 메모리 최적화: ConversationSummaryBufferMemory로 토큰 효율성 개선
# - 사용자 학습: 개인별 이모지 선호도 학습하여 맞춤형 변환

# ⚠️ 주의사항:
# - 메모리 성장: ConversationBufferMemory는 무제한 성장하므로 긴 대화 시 비용 증가
# - 키워드 의존성: 현재는 단순 키워드 매칭으로 라우팅, 더 정교한 NLP 분석 고려 필요
# - 문화적 편향: 이모지 선택의 문화적 적절성, 접근성 고려
# - 에러 처리: 네트워크 오류, API 제한, 잘못된 입력에 대한 robust 처리

=== Challenge 03: 듀얼 체인 메모리 시스템 테스트 ===

🎬 영화 이모지 생성 테스트:
인셉션 → 💭🕰️🔁
어벤져스 → 🦸‍♂️🌌💥
명량 → ⚓️🌊🔥

🧠 메모리 조회 테스트:
첫 번째 영화: 첫 번째로 물어본 영화는 "인셉션"입니다.
마지막 영화: 마지막으로 물어본 영화는 '명량'입니다.
두 번째 영화: 사용자가 두 번째로 물은 영화는 '어벤져스'입니다.

✅ 테스트 완료!
